In [1]:
# Installa le librerie necessarie
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install Pillow
!pip install lime
!pip install tensorflow

# Importa le librerie
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import KMeans
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

2025-10-29 15:21:14.591019: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-29 15:21:14.668639: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 15:21:17.173697: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
import zipfile

# Percorso del file zip
zip_path = "./opere.zip"

# Percorso di estrazione
extract_path = "./opere"

# Estrai il contenuto
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Cartella opere estratta correttamente.")

Cartella opere estratta correttamente.


In [3]:
# Percorso locale alla cartella immagini
IMAGE_DIR = "./opere/opere"
TARGET_SIZE = (224, 224)  # Dimensione standard per VGG16

In [4]:
import os

# Stampa i primi 5 file trovati nella cartella
print("File trovati nella cartella opere:")
print(os.listdir(IMAGE_DIR)[:5])

File trovati nella cartella opere:
['23278.jpg', '32054.jpg', '17014.jpg', '15673.jpg', '79806.jpg']


In [5]:
# Carica il modello VGG16 pre-addestrato
base_model = VGG16(weights='imagenet', include_top=False)

# Estrazione delle feature dallo strato 'block5_pool'
model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)

2025-10-29 15:21:18.325920: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
def extract_features(img_path, model):
    try:
        img = Image.open(img_path).resize(TARGET_SIZE)
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        features = model.predict(img_array, verbose=0)
        features = features.mean(axis=(1, 2)).flatten()
        return features
    except Exception as e:
        print(f"Errore con {img_path}: {e}")
        return None


In [ ]:
all_features = []
image_filenames = []

print("Inizio estrazione delle feature...")

counter = 0  # contatore immagini elaborate

for filename in os.listdir(IMAGE_DIR):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        file_path = os.path.join(IMAGE_DIR, filename)
        features = extract_features(file_path, model)
        if features is not None:
            all_features.append(features)
            image_filenames.append(filename)
            counter += 1

            # Messaggio ogni 100 immagini
            if counter % 100 == 0:
                print(f"{counter} immagini elaborate...")

# Conversione finale in matrice
feature_matrix = np.array(all_features)

print(f"\nEstrazione completata: {len(image_filenames)} immagini.")
print(f"Dimensione matrice delle feature: {feature_matrix.shape}")

Inizio estrazione delle feature...
100 immagini elaborate...
200 immagini elaborate...
300 immagini elaborate...
400 immagini elaborate...
500 immagini elaborate...


In [ ]:
K = 5  # Numero di cluster
print(f"Inizio clustering con K = {K}...")

kmeans_model = KMeans(n_clusters=K, random_state=42, n_init=10)
cluster_labels = kmeans_model.fit_predict(feature_matrix)

print("Clustering completato.")

In [ ]:
results_df = pd.DataFrame({
    'Filename': image_filenames,
    'Cluster': cluster_labels
})

print("Distribuzione dei cluster:")
print(results_df['Cluster'].value_counts())

# Visualizza le prime 5 immagini di un cluster
cluster_id_to_show = 0
cluster_images = results_df[results_df['Cluster'] == cluster_id_to_show]['Filename'].tolist()

fig, axes = plt.subplots(1, min(5, len(cluster_images)), figsize=(15, 5))
fig.suptitle(f"Cluster {cluster_id_to_show}", fontsize=16)

for i, filename in enumerate(cluster_images[:5]):
    img_path = os.path.join(IMAGE_DIR, filename)
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].set_title(filename[:15] + '...')
    axes[i].axis('off')

plt.show()